In [1]:
import json
import pprint
import subprocess

import duckdb
import polars as pl

In [2]:
import os
import sys
import django

# Add your project root to the Python path if necessary
sys.path.append('/home/reepoi/GitHub/Orange-Button-ChecklistRegistry') 

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# Set the DJANGO_SETTINGS_MODULE environment variable
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'checklist.settings')

# Initialize Django
django.setup()

In [3]:
from server import models

### Maintainer

In [4]:
maintainer, was_created = models.ChecklistTemplateMaintainer.objects.update_or_create(
    Email_Value='info@blubanyan.com',
    URL_Value='https://blubanyan.com',
    WorkPhone_Value='510.929.1070',
    ChecklistTemplateMaintainerName_Value='Blu Banyan',
)
f'{maintainer=}, {was_created=}'
# maintainer.save()

'maintainer=<ChecklistTemplateMaintainer: ChecklistTemplateMaintainer object (1)>, was_created=True'

### Upsert AnswerOptions
The AnswerOption table should have a unique constaint on Value, and the columns of this table are id and Value. Value can be treated as an alternate key.

In [5]:
answer_option_groups = pl.read_csv('AnswerOptionGroups.2026-02-11.csv')
answer_option_groups = {
    col.name: [v for v in col if v is not None]
    for col in answer_option_groups
}
pprint.pp(answer_option_groups, compact=True)

{'Home Style': ['Single', '2 Story', '3 Story', 'Mobile', 'Condo/Townhome'],
 'Roof Material': ['Cemented-Down Tile', 'Clay tile', 'Comp Shingle',
                   'Concrete', 'Corrugated Metal', 'Elastomeric', 'Flat Tile',
                   'Foam', 'Gravel', 'Metal Tile',
                   'Not Applicable (No modules on the roof)', 'Rolled Asphalt',
                   'S-Tile', 'Standing Seam Metal', 'TPO',
                   'Unable To Determine', 'W-Tile', 'Wood shake'],
 'Panel Manufacturer': ['Siemens/Murray/ITE/Gould/Crouse Hinds', 'Square D HOM',
                        'Square D QO', 'Eaton BH/Cutler Hammer/Westinghouse',
                        'Eaton BR/Challenger/Bryant',
                        'General Electric/Milbank/Unicorn', 'Zinsco/Sylvania',
                        'Federal Pacific', 'Not accessible'],
 'Main Breaker Rating': ['225A', '400A', '200A', '300A', '175A', '250A', '150A',
                         '125A', '100A', 'No Main Breaker',
                      

In [6]:
for k, v in answer_option_groups.items():
    answer_option_groups[k] = [
        models.AnswerOption.objects.get_or_create(Value=vv)[0] for vv in v
    ]
pprint.pp(answer_option_groups, compact=True)

{'Home Style': [<AnswerOption: Single>, <AnswerOption: 2 Story>,
                <AnswerOption: 3 Story>, <AnswerOption: Mobile>,
                <AnswerOption: Condo/Townhome>],
 'Roof Material': [<AnswerOption: Cemented-Down Tile>,
                   <AnswerOption: Clay tile>, <AnswerOption: Comp Shingle>,
                   <AnswerOption: Concrete>, <AnswerOption: Corrugated Metal>,
                   <AnswerOption: Elastomeric>, <AnswerOption: Flat Tile>,
                   <AnswerOption: Foam>, <AnswerOption: Gravel>,
                   <AnswerOption: Metal Tile>,
                   <AnswerOption: Not Applicable (No modules on the roof)>,
                   <AnswerOption: Rolled Asphalt>, <AnswerOption: S-Tile>,
                   <AnswerOption: Standing Seam Metal>, <AnswerOption: TPO>,
                   <AnswerOption: Unable To Determine>, <AnswerOption: W-Tile>,
                   <AnswerOption: Wood shake>],
 'Panel Manufacturer': [<AnswerOption: Siemens/Murray/ITE/Gould/Crou

### Checklist Templates

In [7]:
checklist_template_questions = pl.read_csv('ChecklistTemplateQuestions.2026-02-11.csv')
checklist_template_questions

Checklist Template Name,DisplaySeqNumber,SectionName,Question Label,Answer Type,For Select there would be 'AnswerOption',Example,Tag,Help Text
str,i64,str,str,str,str,str,str,str
"""Site Survey""",10,"""Ground""","""Cancelled at the Door Y/N""","""EnumeratedSingle""","""Yes / No""",null,"""Residential""","""Indicate if the homeowner comm…"
"""Site Survey""",20,"""Ground""","""Home Type""","""EnumeratedSingle""","""Home Style""",null,"""Residential""","""Define the home type (e.g. sin…"
"""Site Survey""",30,"""Roof""","""Was Layout Provided for Survey""","""EnumeratedSingle""","""Yes / No""",null,"""Residential""","""Contact your supervisor if the…"
"""Site Survey""",40,"""Roof""","""Panels Proposed on Additional …","""EnumeratedSingle""","""Yes / No""",null,"""Residential""","""Indicate if any solar panels a…"
"""Site Survey""",50,"""Roof""","""Roof Material""","""EnumeratedSingle""","""Roof Material""",null,"""Residential""","""Record the type of roofing mat…"
…,…,…,…,…,…,…,…,…
"""Installation""",480,"""Electrical Ground""","""Was A Line Side Tap Completed""","""EnumeratedSingle""","""Yes / No""",null,"""Residential""","""Indicate if a supply-side (lin…"
"""Installation""",490,"""Electrical Ground""","""Was A Derate Completed""","""EnumeratedSingle""","""Yes / No""",null,"""Residential""","""Indicate if the main breaker w…"
"""Installation""",500,"""Electrical Ground""","""Was A Hawaiian Tie In Complete""","""EnumeratedSingle""","""Yes / No""",null,"""Residential""","""Indicate if a Hawaiian-style i…"


In [8]:
checklist_templates = []
for name in checklist_template_questions['Checklist Template Name'].unique():
    checklist_template, was_created = models.ChecklistTemplate.objects.update_or_create(
        ChecklistTemplateName_Value=name,
        ChecklistTemplateVersion_Value='1',
        ChecklistTemplateMaintainer=maintainer,
    )
    print(f'{checklist_template=}, {was_created=}')
    checklist_templates.append(checklist_template)

checklist_template=<ChecklistTemplate: ChecklistTemplate object (1)>, was_created=True
checklist_template=<ChecklistTemplate: ChecklistTemplate object (2)>, was_created=True


##### Questions

In [9]:
for template in checklist_templates:
    questions = checklist_template_questions.filter(pl.col('Checklist Template Name') == template.ChecklistTemplateName_Value)
    question_rows = []
    for row in questions.iter_rows(named=True):
        question = models.Question(
            AnswerType_Value=row['Answer Type'],
            DisplaySeqNumber_Value=row['DisplaySeqNumber'],
            QuestionHelp_Value=row['Help Text'],
            QuestionLabel_Value=row['Question Label'],
            SectionName_Value=row['SectionName'],
        )
        question.save()
        if row['Answer Type'] in ('EnumeratedSingle', 'EnumeratedMultiple'):
            answer_option_group = row["For Select there would be 'AnswerOption'"]
            question.AnswerOptions.set(answer_option_groups[answer_option_group])
        question_rows.append(question)
    template.Questions.set(question_rows)

##### Tags
The Tags table should have a unique constaint on Value, and the columns of this table are id and Value. Value can be treated as an alternate key.

In [10]:
for template in checklist_templates:
    tags = checklist_template_questions.filter(pl.col('Checklist Template Name') == template.ChecklistTemplateName_Value)['Tag'].unique()
    tags = [models.Tag.objects.get_or_create(Value=tag)[0] for tag in tags]
    template.Tags.set(tags)